In [1]:
import torchvision
import torch.nn as nn
import torch.nn.functional as nnf
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor, normalize
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torchvision import transforms, models
from torchvision.datasets import Food101
from torchvision import datasets, transforms
import statistics as st

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

  return correct / total * 100

def train_fn(epochs: int, train_loader: DataLoader,
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer, train_dataset_length):

  # losses = []
  # accuracies = []

  net.to(device)

  for e in range(epochs):
    net.train()
    running_loss = 0.0

    for images, labels in train_loader:
      images, labels = images.to(device), labels.type(torch.LongTensor).to(device)

      optimizer.zero_grad()
      outputs = net(images)
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / train_dataset_length

    print(f"Loss: {epoch_loss}")

    # acc = test_acc(net, test_loader)
    # print(f"Acuratetea la finalul epocii {e + 1} este {acc:.2f}%")

    # losses.append(epoch_loss)
    # accuracies.append(acc)

    # torch.save(net.state_dict(), f'resnet50_multiplefclayers_adamw_epoch{e + 1}.pkl')

  # loss_graph(losses)
  # accuracy_graph(accuracies)

def loss_graph(losses):
    plt.plot(losses)

    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

def accuracy_graph(accuracies):
    plt.plot(accuracies)

    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.show()

class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, 101)
        )

    def forward(self, x):
        return self.classifier(x)

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
# SGD

configs = [(0.9, 0.001), (0.9, 0.0001), (0.9, 0.00001), (0.95, 0.001), (0.95, 0.0001), (0.95, 0.00001), (0.99, 0.001), (0.99, 0.0001), (0.99, 0.00001)]
mean_accuracies = []
i = 1

for momentum, weight_decay in configs:

    print(f'Config {i}:')
    i += 1
    accuracies = []

    for split_number in range(1, 6):

        train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
        val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
    
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        val_dataset = datasets.ImageFolder(val_dir, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        resnet = models.resnet50(pretrained=True)
        resnet = nn.Sequential(*list(resnet.children())[:-1])

        # Freeze all layers except the final classification layer
        for param in resnet.parameters():
            param.requires_grad = False

        resnet = nn.Sequential(
            resnet,
            CustomClassifier()
        )

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=momentum, weight_decay=weight_decay)

        start = time.time()
        print(f'Training split {split_number}')

        train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

        acc = test_acc(resnet, val_loader)

        print(f'Accuracy: {acc:.2f}%')
        accuracies.append(acc)

        end = time.time()
        print(f'Duration of training: {(end - start) / 60}')

    mean_accuracies.append(st.mean(accuracies))
    
    print(accuracies)
    print(st.mean(accuracies))

print(mean_accuracies)

Config 1:


c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training split 1
Loss: 3.0810552119264507
Accuracy: 48.24%
Duration of training: 12.978929408391316
Training split 2
Loss: 3.068732658518423
Accuracy: 47.69%
Duration of training: 12.902049148082734
Training split 3
Loss: 3.0849373537951177
Accuracy: 47.09%
Duration of training: 12.216011277834575
Training split 4
Loss: 3.076727840404699
Accuracy: 48.29%
Duration of training: 12.372770484288534
Training split 5
Loss: 3.0747044070404357
Accuracy: 47.27%
Duration of training: 11.904874269167582
[48.23762376237624, 47.68811881188119, 47.089108910891085, 48.28712871287129, 47.27227722772277]
47.714851485148515
Config 2:
Training split 1
Loss: 3.0778460174031776
Accuracy: 47.93%
Duration of training: 12.07348201672236
Training split 2
Loss: 3.0628571816246106
Accuracy: 47.51%
Duration of training: 12.030071485042573
Training split 3
Loss: 3.0777367574861736
Accuracy: 47.44%
Duration of training: 11.881509733200073
Training split 4
Loss: 3.0871829512567803
Accuracy: 48.14%
Duration of traini

In [2]:
# Adam

accuracies = []

for split_number in range(1, 6):

    train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
    val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
 
    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    resnet = models.resnet50(pretrained=True)
    resnet = nn.Sequential(*list(resnet.children())[:-1])

    # Freeze all layers except the final classification layer
    for param in resnet.parameters():
        param.requires_grad = False

    resnet = nn.Sequential(
        resnet,
        CustomClassifier()
    )

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet.parameters(), lr=0.0009)

    start = time.time()
    print(f'Training split {split_number}')

    train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

    acc = test_acc(resnet, val_loader)

    print(f'Accuracy: {acc:.2f}%')
    accuracies.append(acc)

    end = time.time()
    print(f'Duration of training: {(end - start) / 60}')

print(accuracies)
print(st.mean(accuracies))

c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training split 1
Loss: 2.3789230561492465
Accuracy: 52.08%
Duration of training: 16.35149817466736
Training split 2
Loss: 2.3709249138596036
Accuracy: 51.59%
Duration of training: 15.881305054823558
Training split 3
Loss: 2.3662334962410503
Accuracy: 51.42%
Duration of training: 15.982991123199463
Training split 4
Loss: 2.384864184502328
Accuracy: 51.32%
Duration of training: 15.936261602242787
Training split 5
Loss: 2.3707797448941976
Accuracy: 50.00%
Duration of training: 122.5415296514829
[52.07920792079208, 51.58910891089109, 51.415841584158414, 51.32178217821782, 50.0]
51.28118811881188


In [4]:
# AdamW

configs = [(0.9, 0.999, 0.01), (0.9, 0.999, 0.001), (0.9, 0.999, 0.0001), (0.95, 0.999, 0.01), (0.95, 0.999, 0.001), (0.95, 0.999, 0.0001), (0.85, 0.999, 0.01), (0.85, 0.999, 0.001), (0.85, 0.999, 0.0001)]
mean_accuracies = []
i = 1

for b1, b2, weight_decay in configs:

    print(f'Config {i}:')
    i += 1
    accuracies = []

    for split_number in range(1, 6):

        train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
        val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
    
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        val_dataset = datasets.ImageFolder(val_dir, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        resnet = models.resnet50(pretrained=True)
        resnet = nn.Sequential(*list(resnet.children())[:-1])

        # Freeze all layers except the final classification layer
        for param in resnet.parameters():
            param.requires_grad = False

        resnet = nn.Sequential(
            resnet,
            CustomClassifier()
        )

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(resnet.parameters(), lr=0.001, betas=(b1, b2), weight_decay=weight_decay)

        start = time.time()
        print(f'Training split {split_number}')

        train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

        acc = test_acc(resnet, val_loader)

        print(f'Accuracy: {acc:.2f}%')
        accuracies.append(acc)

        end = time.time()
        print(f'Duration of training: {(end - start) / 60}')

    mean_accuracies.append(st.mean(accuracies))
    
    print(accuracies)
    print(st.mean(accuracies))

print(mean_accuracies)

Config 1:
Training split 1
Loss: 2.3691504467123807
Accuracy: 52.10%
Duration of training: 8.628341674804688
Training split 2
Loss: 2.3708721514975672
Accuracy: 50.81%
Duration of training: 8.635489885012309
Training split 3
Loss: 2.362696132943182
Accuracy: 51.02%
Duration of training: 8.632052675882976
Training split 4
Loss: 2.365888430246032
Accuracy: 52.20%
Duration of training: 8.63960640033086
Training split 5
Loss: 2.3532089421300606
Accuracy: 50.35%
Duration of training: 8.631850492954253
[52.0990099009901, 50.806930693069305, 51.024752475247524, 52.1980198019802, 50.34653465346535]
51.2950495049505
Config 2:
Training split 1
Loss: 2.359435354317769
Accuracy: 52.14%
Duration of training: 8.645898675918579
Training split 2
Loss: 2.361776711208986
Accuracy: 50.94%
Duration of training: 8.63192676305771
Training split 3
Loss: 2.365243058582344
Accuracy: 52.07%
Duration of training: 8.6666472752889
Training split 4
Loss: 2.3650460994833766
Accuracy: 52.17%
Duration of training: 8.6

In [5]:
print(max(mean_accuracies))

51.93366336633664
